In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

In [2]:
# load the MQ features
mq_features_df = pd.read_pickle('/Users/darylwilding-mcbride/Downloads/mq-ms1-rt-3000-3060.pkl')
mq_features_df["hover"] = mq_features_df["mz"].map('{:,.4f} m/z'.format) + ', ' + mq_features_df["scan"].map('{} scan'.format) + ', ' + mq_features_df["rt"].map('RT {:.1f} secs'.format) + ', ' + mq_features_df["msms_scan_number"].map('msms scan number {}'.format)

In [3]:
evidence_df = pd.read_csv('/Users/darylwilding-mcbride/Downloads/maxquant_results/txt/evidence.txt', sep='\t')
evidence_df = evidence_df[(evidence_df['Retention time'] >= 3000) & (evidence_df['Retention time'] <= 3060)]
evidence_df['MS/MS scan number'] = evidence_df['MS/MS scan number'].apply(lambda x: int(x))

In [17]:
evidence_df.columns

Index(['Sequence', 'Length', 'Modifications', 'Modified sequence',
       'Oxidation (M) Probabilities', 'Oxidation (M) Score Diffs',
       'Acetyl (Protein N-term)', 'Oxidation (M)', 'Missed cleavages',
       'Proteins', 'Leading proteins', 'Leading razor protein', 'Type',
       'Raw file', 'MS/MS m/z', 'Charge', 'm/z', 'Mass', 'Resolution',
       'Uncalibrated - Calibrated m/z [ppm]',
       'Uncalibrated - Calibrated m/z [Da]', 'Mass error [ppm]',
       'Mass error [Da]', 'Uncalibrated mass error [ppm]',
       'Uncalibrated mass error [Da]', 'Max intensity m/z 0', 'Retention time',
       'Retention length', 'Calibrated retention time',
       'Calibrated retention time start', 'Calibrated retention time finish',
       'Retention time calibration', 'Ion mobility index',
       'Ion mobility length', 'CCS', 'CCS length', 'Match time difference',
       'Match m/z difference', 'Match q-value', 'Match score',
       'Number of data points', 'Number of scans', 'Number of isotopic

In [4]:
mq_features_df = pd.merge(mq_features_df, evidence_df[['MS/MS scan number','Score']], how='left', left_on=['msms_scan_number'], right_on=['MS/MS scan number'])

In [5]:
mq_features_df.head()

,mz,charge,intensity,monoisotopic_mass,rt,scan,msms_scan_number,hover,MS/MS scan number,Score
0,326.70863,2,7996,651.40270,3017.769,852,1790,"326.7086 m/z, 852 scan, RT 3017.8 secs, msms s...",NaN,NaN
1,364.74869,2,35051,727.48282,3018.951,768,3086,"364.7487 m/z, 768 scan, RT 3019.0 secs, msms s...",NaN,NaN
2,363.87617,3,12509,1088.60670,3029.586,825,20281,"363.8762 m/z, 825 scan, RT 3029.6 secs, msms s...",20281.0,29.31
3,367.21087,3,168590,1098.61080,3040.218,825,20900,"367.2109 m/z, 825 scan, RT 3040.2 secs, msms s...",NaN,NaN
4,371.21264,3,27040,1110.61610,3021.314,771,21731,"371.2126 m/z, 771 scan, RT 3021.3 secs, msms s...",NaN,NaN


In [6]:
mq_features_df.columns

Index(['mz', 'charge', 'intensity', 'monoisotopic_mass', 'rt', 'scan',
       'msms_scan_number', 'hover', 'MS/MS scan number', 'Score'],
      dtype='object')

In [7]:
mq_features_df = mq_features_df[mq_features_df['MS/MS scan number'].notnull() & mq_features_df['Score'].notnull()].copy()

In [8]:
mq_features_df.head()

,mz,charge,intensity,monoisotopic_mass,rt,scan,msms_scan_number,hover,MS/MS scan number,Score
2,363.87617,3,12509,1088.60670,3029.586,825,20281,"363.8762 m/z, 825 scan, RT 3029.6 secs, msms s...",20281.0,29.310
14,417.23776,3,45793,1248.69150,3015.399,777,32256,"417.2378 m/z, 777 scan, RT 3015.4 secs, msms s...",32256.0,29.310
16,415.22893,3,19425,1242.66500,3041.400,795,31776,"415.2289 m/z, 795 scan, RT 3041.4 secs, msms s...",31776.0,37.591
21,422.26385,2,222860,842.51315,3035.494,678,6677,"422.2638 m/z, 678 scan, RT 3035.5 secs, msms s...",6677.0,49.058
24,426.25686,3,307840,1275.74880,3026.043,807,34446,"426.2569 m/z, 807 scan, RT 3026.0 secs, msms s...",34446.0,54.436


In [9]:
# load my features
my_features_df = pd.read_pickle('/Users/darylwilding-mcbride/Downloads/ms1_deduped_df.pkl')
my_features_df["hover"] = my_features_df["monoisotopic_mz"].map('{:,.4f} m/z'.format) + ', ' + my_features_df["scan_apex"].map('{:,.0f} scan'.format) + ', ' + my_features_df["rt_apex"].map('RT {:.1f} secs'.format) + ', ' + my_features_df["feature_id"].map('feature {}'.format)

In [10]:
my_features_df.head(2)

,monoisotopic_mz,charge,intensity,scan_apex,scan_curve_fit,scan_lower,scan_upper,rt_apex,rt_curve_fit,rt_lower,rt_upper,precursor_id,ms2_frames,duplicates,feature_id,hover
0,561.297167,2,1932201,619.69,True,594.68,644.71,3030.82,True,3018.54,3043.09,93005,"[28138, 28140, 28152, 28185, 28196, 28330]",10,1,"561.2972 m/z, 620 scan, RT 3030.8 secs, feature 1"
1,673.392297,2,1875771,541.96,True,516.70,567.22,3018.45,True,3007.41,3029.48,92744,"[28128, 28129, 28132, 28136, 28137, 28208]",5,2,"673.3923 m/z, 542 scan, RT 3018.4 secs, feature 2"


In [11]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # requires version >= 1.9.0

3.5.0


In [12]:
init_notebook_mode(connected=True)

In [13]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]

In [14]:
tableau20_py = tableau20.copy()

In [15]:
import plotly.plotly as py
import plotly.graph_objs as go

traces = []

trace = go.Scatter3d(
    x = my_features_df.monoisotopic_mz,
    y = my_features_df.scan_apex,
    z = my_features_df.rt_apex,
    mode = 'markers',
    marker = dict(
        size = 5,
        color = "rgb(255,128,128)",
        opacity = 0.7
    ),
    text = my_features_df.hover,
    hoverinfo = 'text',
    name = "PASEF-driven"
)
traces.append((trace))

trace = go.Scatter3d(
    x = mq_features_df.mz,
    y = mq_features_df.scan,
    z = mq_features_df.rt,
    mode = 'markers',
    marker = dict(
        size = 5,
        color = "rgb(128,255,128)",
        opacity = 0.7
    ),
    text = mq_features_df.hover,
    hoverinfo = 'text',
    name = "MaxQuant"
)
traces.append((trace))

data = traces
layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    title="Comparing ms1 features",
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=40
    ),
    scene = dict(
        xaxis = dict(
            title='m/z', autorange='reversed'),
        yaxis = dict(
            title='scan'),
        zaxis = dict(
            title='retention time')
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)